# **Directed Unweighted Graphs - Complete Guide**

A **Directed Unweighted Graph** (also called a **Digraph**) is a collection of vertices (nodes) connected by directed edges where edges have a specific direction but no associated weight. This means if there's an edge from vertex A to vertex B, you can only travel from A to B, not necessarily from B to A.

**Key Features:**
- **Vertices (Nodes)**: Individual points or entities in the graph
- **Directed Edges**: Connections with a specific direction (A → B)
- **Asymmetric Relations**: Edge (u,v) does NOT imply edge (v,u) exists
- **In-degree**: Number of edges pointing into a vertex
- **Out-degree**: Number of edges pointing out from a vertex
- **Path**: Sequence of vertices following edge directions
- **Cycle**: Directed path that starts and ends at the same vertex

**Graph Terminology:**
- **Source Vertex**: Vertex with in-degree = 0 (no incoming edges)
- **Sink Vertex**: Vertex with out-degree = 0 (no outgoing edges)
- **Strongly Connected**: Path exists between every ordered pair of vertices
- **Weakly Connected**: Underlying undirected graph is connected
- **Topological Order**: Linear ordering respecting all edge directions
- **Strongly Connected Components (SCCs)**: Maximal strongly connected subgraphs

**Applications:**
- **Task Scheduling**: Dependencies between tasks, project management
- **Web Pages**: Link structure, PageRank algorithm
- **Social Networks**: Following relationships, influence propagation  
- **Compiler Design**: Control flow graphs, dependency analysis
- **Transportation**: One-way streets, flight connections
- **Biology**: Gene regulation networks, food webs

## Node Definition for Directed Graphs

A **Node (Vertex)** in a directed graph represents an entity that can have incoming and outgoing connections. Each node maintains:

- **Node Identity**: Unique identifier (name, number, etc.)
- **Incoming Edges**: Other nodes that point to this node
- **Outgoing Edges**: Nodes that this node points to
- **Traversal State**: Visited status, discovery/finish times, colors (for algorithms)

In directed graphs, we need to carefully track edge directions since the relationship between nodes is asymmetric. We'll represent nodes as simple identifiers and manage their directed connections through graph data structures.

In [1]:
import random
from collections import defaultdict, deque
import sys

# Create a sample directed unweighted graph with 7 nodes
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Define directed edges for our sample graph
# Format: (from_node, to_node) - direction matters!
edges = [
    ('A', 'B'), ('A', 'C'),
    ('B', 'D'), ('B', 'E'), 
    ('C', 'F'),
    ('D', 'G'),
    ('E', 'F'), ('E', 'G'),
    ('F', 'G'),
    ('G', 'A'),  # Creates a cycle
    ('C', 'D')   # Alternative path
]

print("=== Sample Directed Unweighted Graph ===")
print(f"Nodes: {nodes}")
print(f"Directed edges:")
for u, v in edges:
    print(f"  {u} → {v}")

print(f"\nTotal Nodes: {len(nodes)}")
print(f"Total Directed Edges: {len(edges)}")

# Calculate in-degree and out-degree for each node
in_degree = {node: 0 for node in nodes}
out_degree = {node: 0 for node in nodes}

for u, v in edges:
    out_degree[u] += 1
    in_degree[v] += 1

print("Node degrees:")
for node in nodes:
    print(f"  {node}: in-degree={in_degree[node]}, out-degree={out_degree[node]}")

=== Sample Directed Unweighted Graph ===
Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Directed edges:
  A → B
  A → C
  B → D
  B → E
  C → F
  D → G
  E → F
  E → G
  F → G
  G → A
  C → D

Total Nodes: 7
Total Directed Edges: 11
Node degrees:
  A: in-degree=1, out-degree=2
  B: in-degree=1, out-degree=2
  C: in-degree=1, out-degree=2
  D: in-degree=2, out-degree=1
  E: in-degree=1, out-degree=2
  F: in-degree=2, out-degree=1
  G: in-degree=3, out-degree=1


## Graph Representations

Directed graphs require careful handling of edge directions in both representation methods:

### 1. Adjacency Matrix
- **2D array** where `matrix[i][j] = 1` if there's a directed edge from vertex i to vertex j
- **Asymmetric matrix** for directed graphs (matrix[i][j] ≠ matrix[j][i])
- **Row i**: Outgoing edges from vertex i
- **Column j**: Incoming edges to vertex j

### 2. Adjacency List  
- **Dictionary/List** where each vertex stores a list of vertices it points to
- **Only outgoing edges** are stored in the adjacency list
- **Incoming edges** require separate tracking or reverse adjacency list

**Time Complexity Comparison:**

| Operation | Adjacency Matrix | Adjacency List |
|-----------|------------------|----------------|
| Add Edge | O(1) | O(1) |
| Remove Edge | O(1) | O(out-degree) |
| Check Edge | O(1) | O(out-degree) |
| Get Successors | O(V) | O(out-degree) |
| Get Predecessors | O(V) | O(V + E) |
| Space Complexity | O(V²) | O(V + E) |

In [2]:
class DirectedGraph:
    def __init__(self, nodes):
        self.nodes = nodes
        self.node_to_index = {node: i for i, node in enumerate(nodes)}
        self.num_nodes = len(nodes)
        
        # Adjacency Matrix representation
        self.adj_matrix = [[0 for _ in range(self.num_nodes)] for _ in range(self.num_nodes)]
        
        # Adjacency List representation (outgoing edges)
        self.adj_list = defaultdict(list)
        
        # Reverse adjacency list for incoming edges (useful for some algorithms)
        self.reverse_adj_list = defaultdict(list)
    
    def add_edge(self, u, v):
        """Add directed edge from vertex u to vertex v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        
        # Update adjacency matrix
        self.adj_matrix[u_idx][v_idx] = 1
        
        # Update adjacency lists
        self.adj_list[u].append(v)
        self.reverse_adj_list[v].append(u)  # For incoming edges
    
    def display_matrix(self):
        """Display adjacency matrix representation"""
        print("Adjacency Matrix (rows=from, columns=to):")
        print("     ", end="")
        for node in self.nodes:
            print(f"{node:>3}", end="")
        print()
        
        for i, node in enumerate(self.nodes):
            print(f"{node:>2}: ", end="")
            for j in range(self.num_nodes):
                print(f"{self.adj_matrix[i][j]:>2}", end=" ")
            print()
    
    def display_list(self):
        """Display adjacency list representation"""
        print("Adjacency List (outgoing edges):")
        for node in self.nodes:
            successors = self.adj_list[node]
            print(f"{node} → {successors}")
        
        print("\nReverse Adjacency List (incoming edges):")
        for node in self.nodes:
            predecessors = self.reverse_adj_list[node]
            print(f"{node} ← {predecessors}")
    
    def get_in_degree(self, node):
        """Get in-degree of a node"""
        return len(self.reverse_adj_list[node])
    
    def get_out_degree(self, node):
        """Get out-degree of a node"""
        return len(self.adj_list[node])

# Create and populate our sample directed graph
graph = DirectedGraph(nodes)

# Add all directed edges
for u, v in edges:
    graph.add_edge(u, v)

print("=== Graph Representations ===")
graph.display_matrix()
print()
graph.display_list()
print()

print("Node degree analysis:")
for node in nodes:
    in_deg = graph.get_in_degree(node)
    out_deg = graph.get_out_degree(node)
    print(f"{node}: in-degree={in_deg}, out-degree={out_deg}")

=== Graph Representations ===
Adjacency Matrix (rows=from, columns=to):
       A  B  C  D  E  F  G
 A:  0  1  1  0  0  0  0 
 B:  0  0  0  1  1  0  0 
 C:  0  0  0  1  0  1  0 
 D:  0  0  0  0  0  0  1 
 E:  0  0  0  0  0  1  1 
 F:  0  0  0  0  0  0  1 
 G:  1  0  0  0  0  0  0 

Adjacency List (outgoing edges):
A → ['B', 'C']
B → ['D', 'E']
C → ['F', 'D']
D → ['G']
E → ['F', 'G']
F → ['G']
G → ['A']

Reverse Adjacency List (incoming edges):
A ← ['G']
B ← ['A']
C ← ['A']
D ← ['B', 'C']
E ← ['B']
F ← ['C', 'E']
G ← ['D', 'E', 'F']

Node degree analysis:
A: in-degree=1, out-degree=2
B: in-degree=1, out-degree=2
C: in-degree=1, out-degree=2
D: in-degree=2, out-degree=1
E: in-degree=1, out-degree=2
F: in-degree=2, out-degree=1
G: in-degree=3, out-degree=1


## Breadth-First Search (BFS) on Directed Graphs

**BFS** in directed graphs follows outgoing edges only, exploring nodes level by level from a starting vertex. It finds the **shortest path** in terms of number of edges (hops) while respecting edge directions.

**Algorithm Steps:**
1. Start from source vertex, mark it visited
2. Add source to queue
3. While queue is not empty:
   - Dequeue vertex
   - Visit all unvisited successors (nodes reachable via outgoing edges)
   - Mark successors as visited and enqueue them
4. Process vertices in the order they are dequeued

**Time Complexity:** O(V + E) where V is vertices and E is directed edges
**Space Complexity:** O(V) for the queue and visited set

**Use Cases:**
- **Shortest hop distance** in directed networks
- **Reachability analysis** from a source node  
- **Level-order traversal** of directed graphs
- **Web crawling** following link directions

In [3]:
def bfs_directed(graph, start_node):
    """
    Breadth-First Search traversal of directed graph
    Returns BFS order and distances from start node
    """
    visited = set()
    queue = deque([start_node])
    bfs_order = []
    distances = {start_node: 0}
    parent = {start_node: None}
    
    visited.add(start_node)
    
    print(f"Starting BFS from node: {start_node}")
    print("Step by step traversal (following edge directions):")
    
    step = 1
    while queue:
        current_node = queue.popleft()
        bfs_order.append(current_node)
        
        print(f"Step {step}: Visit {current_node} (distance: {distances[current_node]})")
        
        # Get successors (outgoing edges only)
        successors = []
        for successor in graph.adj_list[current_node]:
            if successor not in visited:
                successors.append(successor)
                visited.add(successor)
                queue.append(successor)
                distances[successor] = distances[current_node] + 1
                parent[successor] = current_node
        
        if successors:
            print(f"         Add to queue: {successors}")
        
        print(f"         Queue state: {list(queue)}")
        step += 1
        print()
    
    # Check which nodes are reachable
    reachable = set(bfs_order)
    unreachable = set(graph.nodes) - reachable
    
    return bfs_order, distances, parent, reachable, unreachable

# Perform BFS traversal
bfs_result, bfs_distances, bfs_parent, reachable, unreachable = bfs_directed(graph, 'A')

print("=== BFS Results ===")
print(f"BFS Order: {' → '.join(bfs_result)}")
print(f"Distances from A: {bfs_distances}")
print(f"Parent nodes: {bfs_parent}")
print(f"Reachable from A: {sorted(reachable)}")
if unreachable:
    print(f"Unreachable from A: {sorted(unreachable)}")

# Try BFS from different starting points
print(f"\n=== Reachability Analysis ===")
for start in ['C', 'G']:
    _, _, _, reach, unreach = bfs_directed(graph, start)
    print(f"From {start} - Reachable: {sorted(reach)}, Unreachable: {sorted(unreach) if unreach else 'None'}")

Starting BFS from node: A
Step by step traversal (following edge directions):
Step 1: Visit A (distance: 0)
         Add to queue: ['B', 'C']
         Queue state: ['B', 'C']

Step 2: Visit B (distance: 1)
         Add to queue: ['D', 'E']
         Queue state: ['C', 'D', 'E']

Step 3: Visit C (distance: 1)
         Add to queue: ['F']
         Queue state: ['D', 'E', 'F']

Step 4: Visit D (distance: 2)
         Add to queue: ['G']
         Queue state: ['E', 'F', 'G']

Step 5: Visit E (distance: 2)
         Queue state: ['F', 'G']

Step 6: Visit F (distance: 2)
         Queue state: ['G']

Step 7: Visit G (distance: 3)
         Queue state: []

=== BFS Results ===
BFS Order: A → B → C → D → E → F → G
Distances from A: {'A': 0, 'B': 1, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 3}
Parent nodes: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B', 'E': 'B', 'F': 'C', 'G': 'D'}
Reachable from A: ['A', 'B', 'C', 'D', 'E', 'F', 'G']

=== Reachability Analysis ===
Starting BFS from node: C
Step by step trav

## Depth-First Search (DFS) on Directed Graphs

**DFS** in directed graphs explores as far as possible along each directed path before backtracking. It's particularly useful for **cycle detection** and **topological sorting**.

### DFS Variants for Directed Graphs:

### 1. DFS Traversal (Standard)
- **Explores reachable nodes** following edge directions
- **Detects back edges** which indicate cycles

### 2. DFS with Timestamps
- **Discovery time**: When node is first visited
- **Finish time**: When all descendants are processed
- **Used for topological sorting** and SCC algorithms

### 3. DFS Forest
- **Multiple DFS trees** if graph is not strongly connected
- **White/Gray/Black coloring** for cycle detection

**Time Complexity:** O(V + E) 
**Space Complexity:** O(V) for recursion stack and visited tracking

**Use Cases:**
- **Cycle detection** in directed graphs
- **Topological sorting** of DAGs
- **Strongly connected components**
- **Path finding** with backtracking

In [4]:
def dfs_directed(graph, start_node, visited=None, order=None, timestamps=None, time_counter=None):
    """
    DFS traversal of directed graph with timestamp tracking
    """
    if visited is None:
        visited = set()
        order = []
        timestamps = {}
        time_counter = [0]
        print(f"Starting DFS from node: {start_node}")
        print("Step by step traversal (following edge directions):")
    
    # Mark as visited and record discovery time
    visited.add(start_node)
    time_counter[0] += 1
    discovery_time = time_counter[0]
    timestamps[start_node] = {'discovery': discovery_time, 'finish': None}
    
    print(f"Time {discovery_time}: Discover {start_node}")
    
    # Visit all unvisited successors
    for successor in sorted(graph.adj_list[start_node]):  # Sort for consistent output
        if successor not in visited:
            print(f"         Recursively visit {successor}")
            dfs_directed(graph, successor, visited, order, timestamps, time_counter)
    
    # Record finish time and add to order
    time_counter[0] += 1
    finish_time = time_counter[0]
    timestamps[start_node]['finish'] = finish_time
    order.append(start_node)
    
    print(f"Time {finish_time}: Finish {start_node}")
    
    return order, timestamps

def dfs_forest(graph):
    """
    Perform DFS on entire graph (DFS Forest)
    Handles disconnected components
    """
    visited = set()
    forest = []
    all_timestamps = {}
    time_counter = [0]
    
    print("=== DFS Forest (Complete Graph Traversal) ===")
    
    for node in graph.nodes:
        if node not in visited:
            print(f"\nStarting new DFS tree from: {node}")
            tree_order, tree_timestamps = dfs_directed(graph, node, visited, [], {}, time_counter)
            forest.extend(tree_order)
            all_timestamps.update(tree_timestamps)
    
    return forest, all_timestamps

def detect_cycle_dfs(graph):
    """
    Detect cycles in directed graph using DFS coloring
    WHITE (0) = unvisited, GRAY (1) = processing, BLACK (2) = finished
    """
    color = {node: 0 for node in graph.nodes}  # 0=WHITE, 1=GRAY, 2=BLACK
    parent = {node: None for node in graph.nodes}
    cycle_edges = []
    
    def dfs_visit(node):
        color[node] = 1  # Mark as GRAY (processing)
        
        for successor in graph.adj_list[node]:
            if color[successor] == 1:  # Back edge found (cycle)
                cycle_edges.append((node, successor))
                return True
            elif color[successor] == 0 and dfs_visit(successor):
                return True
        
        color[node] = 2  # Mark as BLACK (finished)
        return False
    
    print("=== Cycle Detection using DFS ===")
    has_cycle = False
    
    for node in graph.nodes:
        if color[node] == 0:
            if dfs_visit(node):
                has_cycle = True
    
    print(f"Graph has cycle: {has_cycle}")
    if cycle_edges:
        print(f"Back edges (indicating cycles): {cycle_edges}")
    
    return has_cycle, cycle_edges

# Perform DFS traversal
print("=== DFS Traversal ===")
dfs_order, dfs_timestamps = dfs_directed(graph, 'A')
print(f"\nDFS Order (finish times): {' → '.join(dfs_order)}")
print(f"Timestamps: {dfs_timestamps}")

# Perform DFS Forest
dfs_forest_order, all_timestamps = dfs_forest(graph)

print(f"\n=== DFS Forest Results ===")
print(f"Complete DFS order: {' → '.join(dfs_forest_order)}")

# Detect cycles
has_cycle, back_edges = detect_cycle_dfs(graph)

=== DFS Traversal ===
Starting DFS from node: A
Step by step traversal (following edge directions):
Time 1: Discover A
         Recursively visit B
Time 2: Discover B
         Recursively visit D
Time 3: Discover D
         Recursively visit G
Time 4: Discover G
Time 5: Finish G
Time 6: Finish D
         Recursively visit E
Time 7: Discover E
         Recursively visit F
Time 8: Discover F
Time 9: Finish F
Time 10: Finish E
Time 11: Finish B
         Recursively visit C
Time 12: Discover C
Time 13: Finish C
Time 14: Finish A

DFS Order (finish times): G → D → F → E → B → C → A
Timestamps: {'A': {'discovery': 1, 'finish': 14}, 'B': {'discovery': 2, 'finish': 11}, 'D': {'discovery': 3, 'finish': 6}, 'G': {'discovery': 4, 'finish': 5}, 'E': {'discovery': 7, 'finish': 10}, 'F': {'discovery': 8, 'finish': 9}, 'C': {'discovery': 12, 'finish': 13}}
=== DFS Forest (Complete Graph Traversal) ===

Starting new DFS tree from: A
Time 1: Discover A
         Recursively visit B
Time 2: Discover B
  

## Topological Sorting

**Topological Sorting** produces a linear ordering of vertices such that for every directed edge (u,v), vertex u comes before vertex v in the ordering. This is only possible for **Directed Acyclic Graphs (DAGs)**.

### Two Main Algorithms:

### 1. DFS-based Topological Sort
- **Run DFS** and order vertices by **decreasing finish times**
- **Reverse postorder** gives topological order
- **O(V + E)** time complexity

### 2. Kahn's Algorithm (BFS-based)
- **Start with vertices having in-degree 0**
- **Remove vertices** and update in-degrees
- **Use queue** to process vertices

**Applications:**
- **Task scheduling** with dependencies
- **Course prerequisites** ordering
- **Build systems** (compile dependencies)
- **Spreadsheet formula** evaluation

**Time Complexity:** O(V + E) for both algorithms
**Space Complexity:** O(V) for auxiliary data structures

In [5]:
def topological_sort_dfs(graph):
    """
    Topological sort using DFS (works only on DAGs)
    Returns topological order or None if cycle exists
    """
    # First check if graph is acyclic
    has_cycle, _ = detect_cycle_dfs(graph)
    if has_cycle:
        print("Cannot perform topological sort: Graph contains cycles")
        return None
    
    visited = set()
    topo_stack = []
    
    def dfs_topo(node, step_counter):
        visited.add(node)
        print(f"Step {step_counter[0]}: Visit {node}")
        step_counter[0] += 1
        
        for successor in sorted(graph.adj_list[node]):
            if successor not in visited:
                dfs_topo(successor, step_counter)
        
        topo_stack.append(node)
        print(f"         Add {node} to stack (finished processing)")
    
    print("=== Topological Sort using DFS ===")
    step_counter = [1]
    
    for node in graph.nodes:
        if node not in visited:
            print(f"Starting DFS from {node}")
            dfs_topo(node, step_counter)
            print()
    
    # Reverse stack to get topological order
    topo_order = topo_stack[::-1]
    return topo_order

def topological_sort_kahn(graph):
    """
    Topological sort using Kahn's Algorithm (BFS-based)
    Returns topological order or None if cycle exists
    """
    # Calculate in-degrees
    in_degree = {node: 0 for node in graph.nodes}
    for node in graph.nodes:
        for successor in graph.adj_list[node]:
            in_degree[successor] += 1
    
    # Find nodes with in-degree 0
    queue = deque([node for node in graph.nodes if in_degree[node] == 0])
    topo_order = []
    
    print("=== Topological Sort using Kahn's Algorithm ===")
    print(f"Initial in-degrees: {in_degree}")
    print(f"Starting with zero in-degree nodes: {list(queue)}")
    print("Step by step execution:")
    
    step = 1
    while queue:
        current = queue.popleft()
        topo_order.append(current)
        
        print(f"Step {step}: Process {current}")
        
        # Reduce in-degree of successors
        new_zero_nodes = []
        for successor in graph.adj_list[current]:
            in_degree[successor] -= 1
            if in_degree[successor] == 0:
                queue.append(successor)
                new_zero_nodes.append(successor)
        
        if new_zero_nodes:
            print(f"         New zero in-degree nodes: {new_zero_nodes}")
        
        print(f"         Current order: {topo_order}")
        step += 1
    
    # Check if all nodes are included (no cycles)
    if len(topo_order) != len(graph.nodes):
        print("Graph contains cycles - topological sort impossible")
        return None
    
    return topo_order

# Create a DAG for topological sorting (remove cycle-creating edge)
print("=== Creating DAG for Topological Sort ===")
dag_edges = [(u, v) for u, v in edges if not (u == 'G' and v == 'A')]  # Remove cycle
print(f"DAG edges (removed G→A): {dag_edges}")

dag = DirectedGraph(nodes)
for u, v in dag_edges:
    dag.add_edge(u, v)

# Test both topological sorting algorithms
print("\n=== Testing on Original Graph (with cycle) ===")
topo_dfs = topological_sort_dfs(graph)

print("\n=== Testing on DAG (cycle removed) ===")
topo_dfs_dag = topological_sort_dfs(dag)
print(f"DFS Topological Order: {topo_dfs_dag}")

print()
topo_kahn_dag = topological_sort_kahn(dag)
print(f"Kahn's Topological Order: {topo_kahn_dag}")

print(f"\n=== Verification ===")
if topo_dfs_dag and topo_kahn_dag:
    print(f"Both algorithms found valid orderings:")
    print(f"DFS result:  {' → '.join(topo_dfs_dag)}")
    print(f"Kahn result: {' → '.join(topo_kahn_dag)}")
    
    # Verify topological property
    print(f"Verifying topological property (all edges go forward):")
    for order_name, order in [("DFS", topo_dfs_dag), ("Kahn", topo_kahn_dag)]:
        position = {node: i for i, node in enumerate(order)}
        valid = True
        for u, v in dag_edges:
            if position[u] >= position[v]:
                print(f"  {order_name}: INVALID - edge {u}→{v} violates order")
                valid = False
        if valid:
            print(f"  {order_name}: VALID topological order")

=== Creating DAG for Topological Sort ===
DAG edges (removed G→A): [('A', 'B'), ('A', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'F'), ('D', 'G'), ('E', 'F'), ('E', 'G'), ('F', 'G'), ('C', 'D')]

=== Testing on Original Graph (with cycle) ===
=== Cycle Detection using DFS ===
Graph has cycle: True
Back edges (indicating cycles): [('G', 'A'), ('F', 'G'), ('E', 'F')]
Cannot perform topological sort: Graph contains cycles

=== Testing on DAG (cycle removed) ===
=== Cycle Detection using DFS ===
Graph has cycle: False
=== Topological Sort using DFS ===
Starting DFS from A
Step 1: Visit A
Step 2: Visit B
Step 3: Visit D
Step 4: Visit G
         Add G to stack (finished processing)
         Add D to stack (finished processing)
Step 5: Visit E
Step 6: Visit F
         Add F to stack (finished processing)
         Add E to stack (finished processing)
         Add B to stack (finished processing)
Step 7: Visit C
         Add C to stack (finished processing)
         Add A to stack (finished processing

## Strongly Connected Components (SCCs)

**Strongly Connected Components** are maximal sets of vertices where every vertex is reachable from every other vertex in the set via directed paths.

### Kosaraju's Algorithm:
1. **Run DFS on original graph** and get finish times
2. **Create transpose graph** (reverse all edge directions)
3. **Run DFS on transpose** in decreasing order of finish times
4. **Each DFS tree** in step 3 is one SCC

### Tarjan's Algorithm:
- **Single DFS pass** with low-link values
- **Stack-based** approach for SCC identification
- **More efficient** but more complex implementation

**Properties of SCCs:**
- **Partition**: Every vertex belongs to exactly one SCC
- **Condensation graph**: SCCs form a DAG when compressed
- **Applications**: Web page classification, circuit analysis

**Time Complexity:** O(V + E) for both algorithms
**Space Complexity:** O(V) for auxiliary data structures

**Use Cases:**
- **Web graph analysis**: Finding strongly connected web communities
- **Social networks**: Detecting mutual influence groups
- **Compiler optimization**: Loop detection and optimization
- **Circuit design**: Analyzing feedback loops

In [6]:
def kosaraju_scc(graph):
    """
    Find Strongly Connected Components using Kosaraju's Algorithm
    """
    # Step 1: Run DFS on original graph to get finish times
    visited = set()
    finish_stack = []
    
    def dfs1(node):
        visited.add(node)
        for successor in graph.adj_list[node]:
            if successor not in visited:
                dfs1(successor)
        finish_stack.append(node)
    
    print("=== Kosaraju's Algorithm for SCCs ===")
    print("Step 1: DFS on original graph")
    
    for node in graph.nodes:
        if node not in visited:
            print(f"  Starting DFS from {node}")
            dfs1(node)
    
    print(f"  Finish order: {finish_stack}")
    
    # Step 2: Create transpose graph (reverse all edges)
    transpose = DirectedGraph(graph.nodes)
    for node in graph.nodes:
        for successor in graph.adj_list[node]:
            transpose.add_edge(successor, node)  # Reverse the edge
    
    print("\nStep 2: Created transpose graph")
    print("Transpose adjacency list:")
    for node in graph.nodes:
        print(f"  {node} → {transpose.adj_list[node]}")
    
    # Step 3: DFS on transpose in reverse finish order
    visited2 = set()
    sccs = []
    
    def dfs2(node, current_scc):
        visited2.add(node)
        current_scc.append(node)
        for successor in transpose.adj_list[node]:
            if successor not in visited2:
                dfs2(successor, current_scc)
    
    print(f"\nStep 3: DFS on transpose graph in reverse finish order")
    
    scc_id = 1
    while finish_stack:
        node = finish_stack.pop()
        if node not in visited2:
            current_scc = []
            print(f"  SCC {scc_id}: Starting from {node}")
            dfs2(node, current_scc)
            sccs.append(current_scc)
            print(f"    Found SCC: {current_scc}")
            scc_id += 1
    
    return sccs

def tarjan_scc(graph):
    """
    Find Strongly Connected Components using Tarjan's Algorithm
    """
    index_counter = [0]
    stack = []
    lowlinks = {}
    index = {}
    on_stack = {}
    sccs = []
    
    def strongconnect(node):
        # Set the depth index for node to the smallest unused index
        index[node] = index_counter[0]
        lowlinks[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
        on_stack[node] = True
        
        # Consider successors of node
        for successor in graph.adj_list[node]:
            if successor not in index:
                # Successor has not yet been visited; recurse on it
                strongconnect(successor)
                lowlinks[node] = min(lowlinks[node], lowlinks[successor])
            elif on_stack[successor]:
                # Successor is in stack and hence in the current SCC
                lowlinks[node] = min(lowlinks[node], index[successor])
        
        # If node is a root node, pop the stack and create an SCC
        if lowlinks[node] == index[node]:
            scc = []
            while True:
                w = stack.pop()
                on_stack[w] = False
                scc.append(w)
                if w == node:
                    break
            sccs.append(scc)
    
    print("=== Tarjan's Algorithm for SCCs ===")
    
    for node in graph.nodes:
        if node not in index:
            strongconnect(node)
    
    return sccs

def analyze_sccs(graph, sccs):
    """
    Analyze the structure of strongly connected components
    """
    print(f"\n=== SCC Analysis ===")
    print(f"Number of SCCs: {len(sccs)}")
    
    # Create SCC mapping
    node_to_scc = {}
    for i, scc in enumerate(sccs):
        for node in scc:
            node_to_scc[node] = i
    
    # Analyze SCC sizes
    scc_sizes = [len(scc) for scc in sccs]
    print(f"SCC sizes: {scc_sizes}")
    
    for i, scc in enumerate(sccs):
        if len(scc) == 1:
            node = scc[0]
            # Check if it's a trivial SCC (no self-loop)
            has_self_loop = node in graph.adj_list[node]
            scc_type = "non-trivial (self-loop)" if has_self_loop else "trivial"
            print(f"SCC {i}: {scc} - {scc_type}")
        else:
            print(f"SCC {i}: {scc} - non-trivial (multiple nodes)")
    
    # Create condensation graph (SCC graph)
    condensation_edges = set()
    for node in graph.nodes:
        for successor in graph.adj_list[node]:
            scc_u = node_to_scc[node]
            scc_v = node_to_scc[successor]
            if scc_u != scc_v:  # Edge between different SCCs
                condensation_edges.add((scc_u, scc_v))
    
    print(f"\nCondensation graph edges: {list(condensation_edges)}")
    print("(Condensation graph is always a DAG)")

# Run both SCC algorithms
kosaraju_sccs = kosaraju_scc(graph)
print(f"\nKosaraju SCCs: {kosaraju_sccs}")

print()
tarjan_sccs = tarjan_scc(graph)
print(f"Tarjan SCCs: {tarjan_sccs}")

# Analyze the SCCs
analyze_sccs(graph, kosaraju_sccs)

# Verify both algorithms found same SCCs (possibly in different order)
print(f"\n=== Algorithm Comparison ===")
kosaraju_sets = [set(scc) for scc in kosaraju_sccs]
tarjan_sets = [set(scc) for scc in tarjan_sccs]

kosaraju_sets.sort(key=len, reverse=True)
tarjan_sets.sort(key=len, reverse=True)

print(f"Same SCCs found: {kosaraju_sets == tarjan_sets}")

=== Kosaraju's Algorithm for SCCs ===
Step 1: DFS on original graph
  Starting DFS from A
  Finish order: ['G', 'D', 'F', 'E', 'B', 'C', 'A']

Step 2: Created transpose graph
Transpose adjacency list:
  A → ['G']
  B → ['A']
  C → ['A']
  D → ['B', 'C']
  E → ['B']
  F → ['C', 'E']
  G → ['D', 'E', 'F']

Step 3: DFS on transpose graph in reverse finish order
  SCC 1: Starting from A
    Found SCC: ['A', 'G', 'D', 'B', 'C', 'E', 'F']

Kosaraju SCCs: [['A', 'G', 'D', 'B', 'C', 'E', 'F']]

=== Tarjan's Algorithm for SCCs ===
Tarjan SCCs: [['C', 'F', 'E', 'G', 'D', 'B', 'A']]

=== SCC Analysis ===
Number of SCCs: 1
SCC sizes: [7]
SCC 0: ['A', 'G', 'D', 'B', 'C', 'E', 'F'] - non-trivial (multiple nodes)

Condensation graph edges: []
(Condensation graph is always a DAG)

=== Algorithm Comparison ===
Same SCCs found: True


## Shortest Path in Directed Unweighted Graphs

In **unweighted directed graphs**, **BFS** naturally finds the shortest paths in terms of number of edges. Since all edges have the same "weight" (1), BFS guarantees optimal shortest paths.

**Single-Source Shortest Paths:**
- **BFS from source** finds shortest distances to all reachable nodes
- **Path reconstruction** using parent pointers
- **Time Complexity:** O(V + E)

**All-Pairs Shortest Paths:**
- **Run BFS from each vertex** for complete distance matrix
- **Time Complexity:** O(V × (V + E)) = O(V² + VE)
- **Alternative**: Modified Floyd-Warshall for unweighted graphs

**Applications:**
- **Social networks**: Degrees of separation
- **Web graphs**: Click distance between pages
- **Game AI**: Move count for optimal paths
- **Network analysis**: Hop count in routing

In [7]:
def shortest_paths_bfs(graph, source):
    """
    Find shortest paths from source to all other nodes using BFS
    Returns distances and parent pointers for path reconstruction
    """
    distances = {node: float('inf') for node in graph.nodes}
    parent = {node: None for node in graph.nodes}
    visited = set()
    queue = deque([source])
    
    distances[source] = 0
    visited.add(source)
    
    print(f"Finding shortest paths from {source} using BFS:")
    
    while queue:
        current = queue.popleft()
        
        for successor in graph.adj_list[current]:
            if successor not in visited:
                distances[successor] = distances[current] + 1
                parent[successor] = current
                visited.add(successor)
                queue.append(successor)
                
                print(f"  {source} → {successor}: distance = {distances[successor]} via {current}")
    
    return distances, parent

def reconstruct_path(parent, source, target):
    """Reconstruct shortest path from source to target"""
    if parent[target] is None and target != source:
        return None  # No path exists
    
    path = []
    current = target
    while current is not None:
        path.append(current)
        current = parent[current]
    
    return path[::-1]  # Reverse to get source-to-target path

def all_pairs_shortest_paths(graph):
    """
    Find shortest paths between all pairs of nodes
    Returns distance matrix and path information
    """
    n = len(graph.nodes)
    distances = [[float('inf')] * n for _ in range(n)]
    next_node = [[None] * n for _ in range(n)]
    
    # Initialize distances
    for i in range(n):
        distances[i][i] = 0  # Distance to self is 0
    
    # Set direct edge distances to 1
    for i, u in enumerate(graph.nodes):
        for v in graph.adj_list[u]:
            j = graph.node_to_index[v]
            distances[i][j] = 1
            next_node[i][j] = j
    
    # Floyd-Warshall for unweighted graphs
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if distances[i][k] + distances[k][j] < distances[i][j]:
                    distances[i][j] = distances[i][k] + distances[k][j]
                    next_node[i][j] = next_node[i][k]
    
    return distances, next_node

def print_distance_matrix(distances, nodes):
    """Print distance matrix with node labels"""
    n = len(nodes)
    print("Distance Matrix:")
    print("     ", end="")
    for node in nodes:
        print(f"{node:>4}", end="")
    print()
    
    for i in range(n):
        print(f"{nodes[i]:>2}: ", end="")
        for j in range(n):
            val = distances[i][j]
            if val == float('inf'):
                print("  ∞", end="")
            else:
                print(f"{val:>3}", end="")
        print()

# Test shortest paths from different sources
print("=== Single-Source Shortest Paths ===")

for source in ['A', 'C']:
    print(f"\nFrom source {source}:")
    dist, parent = shortest_paths_bfs(graph, source)
    
    print(f"Distances: {dist}")
    
    # Show paths to all reachable nodes
    print("Paths:")
    for target in graph.nodes:
        if target != source and dist[target] != float('inf'):
            path = reconstruct_path(parent, source, target)
            print(f"  {source} → {target}: {' → '.join(path)} (length: {dist[target]})")
        elif target != source:
            print(f"  {source} → {target}: No path exists")

# Test all-pairs shortest paths
print(f"\n=== All-Pairs Shortest Paths ===")
all_distances, all_next = all_pairs_shortest_paths(graph)
print_distance_matrix(all_distances, graph.nodes)

# Find diameter and radius
print(f"\n=== Graph Properties ===")
finite_distances = []
for i in range(len(graph.nodes)):
    for j in range(len(graph.nodes)):
        if i != j and all_distances[i][j] != float('inf'):
            finite_distances.append(all_distances[i][j])

if finite_distances:
    diameter = max(finite_distances)
    radius = min(max(row) for row in all_distances if max(row) != float('inf'))
    avg_distance = sum(finite_distances) / len(finite_distances)
    
    print(f"Graph diameter: {diameter} (longest shortest path)")
    print(f"Graph radius: {radius}")
    print(f"Average path length: {avg_distance:.2f}")
    print(f"Connected pairs: {len(finite_distances)} out of {len(graph.nodes) * (len(graph.nodes) - 1)}")
else:
    print("Graph is completely disconnected")

=== Single-Source Shortest Paths ===

From source A:
Finding shortest paths from A using BFS:
  A → B: distance = 1 via A
  A → C: distance = 1 via A
  A → D: distance = 2 via B
  A → E: distance = 2 via B
  A → F: distance = 2 via C
  A → G: distance = 3 via D
Distances: {'A': 0, 'B': 1, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 3}
Paths:
  A → B: A → B (length: 1)
  A → C: A → C (length: 1)
  A → D: A → B → D (length: 2)
  A → E: A → B → E (length: 2)
  A → F: A → C → F (length: 2)
  A → G: A → B → D → G (length: 3)

From source C:
Finding shortest paths from C using BFS:
  C → F: distance = 1 via C
  C → D: distance = 1 via C
  C → G: distance = 2 via F
  C → A: distance = 3 via G
  C → B: distance = 4 via A
  C → E: distance = 5 via B
Distances: {'A': 3, 'B': 4, 'C': 0, 'D': 1, 'E': 5, 'F': 1, 'G': 2}
Paths:
  C → A: C → F → G → A (length: 3)
  C → B: C → F → G → A → B (length: 4)
  C → D: C → D (length: 1)
  C → E: C → F → G → A → B → E (length: 5)
  C → F: C → F (length: 1)
  C → G: C

## Cycle Detection Algorithms

**Cycle detection** in directed graphs is crucial for many applications. There are several approaches with different use cases:

### 1. DFS-based Cycle Detection
- **White/Gray/Black coloring** approach
- **Detects back edges** which indicate cycles
- **O(V + E)** time complexity

### 2. Topological Sort Approach
- **Kahn's algorithm** fails if cycles exist
- **Count processed vertices** vs total vertices
- **O(V + E)** time complexity

### 3. Union-Find (for simple cycles)
- **Less common** for directed graphs
- **More suitable** for undirected graphs

**Applications:**
- **Deadlock detection** in operating systems
- **Circular dependency** checking in build systems
- **Infinite loop** prevention in web crawlers
- **Prerequisite validation** in course planning

**Time Complexity:** O(V + E) for all practical algorithms
**Space Complexity:** O(V) for auxiliary data structures

In [8]:
def advanced_cycle_detection(graph):
    """
    Advanced cycle detection with detailed analysis
    Returns cycles found and their classification
    """
    WHITE, GRAY, BLACK = 0, 1, 2
    color = {node: WHITE for node in graph.nodes}
    parent = {node: None for node in graph.nodes}
    cycles = []
    back_edges = []
    
    def dfs_cycle(node, path, path_set):
        color[node] = GRAY
        path.append(node)
        path_set.add(node)
        
        for successor in graph.adj_list[node]:
            if color[successor] == GRAY:  # Back edge (cycle found)
                # Find cycle in current path
                cycle_start_idx = path.index(successor)
                cycle = path[cycle_start_idx:] + [successor]
                cycles.append(cycle)
                back_edges.append((node, successor))
                
                print(f"  Cycle found: {' → '.join(cycle)}")
                
            elif color[successor] == WHITE:
                parent[successor] = node
                if dfs_cycle(successor, path, path_set):
                    return True
        
        color[node] = BLACK
        path.pop()
        path_set.remove(node)
        return len(cycles) > 0
    
    print("=== Advanced Cycle Detection ===")
    
    for node in graph.nodes:
        if color[node] == WHITE:
            print(f"Starting DFS from {node}")
            dfs_cycle(node, [], set())
    
    return cycles, back_edges

def find_all_cycles(graph):
    """
    Find all simple cycles in the directed graph using Johnson's algorithm (simplified)
    """
    def dfs_all_cycles(start, current, path, visited, all_cycles):
        if current == start and len(path) > 1:
            all_cycles.append(path[:])
            return
        
        if current in visited:
            return
        
        visited.add(current)
        
        for successor in graph.adj_list[current]:
            if successor == start or successor not in visited:
                path.append(successor)
                dfs_all_cycles(start, successor, path, visited.copy(), all_cycles)
                path.pop()
    
    print("=== Finding All Simple Cycles ===")
    all_cycles = []
    
    for start_node in graph.nodes:
        visited = set()
        path = [start_node]
        dfs_all_cycles(start_node, start_node, path, visited, all_cycles)
    
    # Remove duplicates and filter valid cycles
    unique_cycles = []
    seen_cycles = set()
    
    for cycle in all_cycles:
        if len(cycle) >= 2:  # Valid cycle must have at least 2 nodes
            # Normalize cycle representation (start from lexicographically smallest)
            min_idx = cycle.index(min(cycle))
            normalized = cycle[min_idx:] + cycle[:min_idx]
            cycle_tuple = tuple(normalized)
            
            if cycle_tuple not in seen_cycles:
                seen_cycles.add(cycle_tuple)
                unique_cycles.append(normalized)
    
    return unique_cycles

def cycle_detection_comparison(graph):
    """
    Compare different cycle detection methods
    """
    print("=== Cycle Detection Method Comparison ===")
    
    # Method 1: DFS coloring
    print("\n1. DFS Coloring Method:")
    cycles_dfs, back_edges = advanced_cycle_detection(graph)
    
    # Method 2: Topological sort (Kahn's algorithm)
    print("\n2. Topological Sort Method:")
    in_degree = {node: 0 for node in graph.nodes}
    for node in graph.nodes:
        for successor in graph.adj_list[node]:
            in_degree[successor] += 1
    
    queue = deque([node for node in graph.nodes if in_degree[node] == 0])
    processed = 0
    
    while queue:
        current = queue.popleft()
        processed += 1
        
        for successor in graph.adj_list[current]:
            in_degree[successor] -= 1
            if in_degree[successor] == 0:
                queue.append(successor)
    
    has_cycle_topo = processed < len(graph.nodes)
    print(f"  Processed {processed}/{len(graph.nodes)} nodes")
    print(f"  Cycle detected: {has_cycle_topo}")
    
    # Method 3: Find all cycles
    print("\n3. All Cycles Detection:")
    all_cycles = find_all_cycles(graph)
    print(f"  Found {len(all_cycles)} simple cycles:")
    for i, cycle in enumerate(all_cycles, 1):
        print(f"    Cycle {i}: {' → '.join(cycle)}")
    
    print(f"\n=== Summary ===")
    print(f"DFS method found {len(cycles_dfs)} cycles")
    print(f"Topological sort detected cycles: {has_cycle_topo}")
    print(f"All cycles method found {len(all_cycles)} simple cycles")
    print(f"Back edges: {back_edges}")

# Run comprehensive cycle detection
cycle_detection_comparison(graph)

# Test on acyclic graph (DAG)
print(f"\n" + "="*60)
print("=== Testing on DAG (Acyclic Graph) ===")
dag_edges = [(u, v) for u, v in edges if not (u == 'G' and v == 'A')]
dag_graph = DirectedGraph(nodes)
for u, v in dag_edges:
    dag_graph.add_edge(u, v)

print("DAG edges (removed cycle-creating edge):")
for u, v in dag_edges:
    print(f"  {u} → {v}")

cycle_detection_comparison(dag_graph)

=== Cycle Detection Method Comparison ===

1. DFS Coloring Method:
=== Advanced Cycle Detection ===
Starting DFS from A
  Cycle found: A → B → D → G → A
Starting DFS from C
Starting DFS from E

2. Topological Sort Method:
  Processed 0/7 nodes
  Cycle detected: True

3. All Cycles Detection:
=== Finding All Simple Cycles ===
  Found 21 simple cycles:
    Cycle 1: A → B → D → G → A
    Cycle 2: A → B → E → F → G → A
    Cycle 3: A → B → E → G → A
    Cycle 4: A → C → F → G → A
    Cycle 5: A → C → D → G → A
    Cycle 6: A → B → B → D → G
    Cycle 7: A → B → B → E → F → G
    Cycle 8: A → B → B → E → G
    Cycle 9: A → C → C → F → G
    Cycle 10: A → C → C → D → G
    Cycle 11: A → B → D → D → G
    Cycle 12: A → C → D → D → G
    Cycle 13: A → B → E → E → F → G
    Cycle 14: A → B → E → E → G
    Cycle 15: A → B → E → F → F → G
    Cycle 16: A → C → F → F → G
    Cycle 17: A → B → D → G → G
    Cycle 18: A → B → E → F → G → G
    Cycle 19: A → B → E → G → G
    Cycle 20: A → C → F → G 

## Algorithm Complexity and Performance Analysis

Let's analyze the time and space complexity of all algorithms implemented for directed graphs:

### Time Complexity Summary

| Algorithm | Best Case | Average Case | Worst Case | Space Complexity |
|-----------|-----------|--------------|------------|------------------|
| **BFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **DFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Topological Sort (DFS)** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Topological Sort (Kahn)** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Kosaraju SCC** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Tarjan SCC** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Cycle Detection** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Shortest Path (BFS)** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **All-Pairs Shortest** | O(V³) | O(V³) | O(V³) | O(V²) |

### Graph Representation Complexity

| Representation | Space | Add Edge | Remove Edge | Check Edge | Get Successors | Get Predecessors |
|----------------|-------|----------|-------------|------------|----------------|------------------|
| **Adjacency Matrix** | O(V²) | O(1) | O(1) | O(1) | O(V) | O(V) |
| **Adjacency List** | O(V + E) | O(1) | O(out-degree) | O(out-degree) | O(out-degree) | O(V + E) |

### Performance Characteristics by Graph Type

**Dense Directed Graphs (E ≈ V²):**
- **Adjacency Matrix** representation becomes space-efficient
- **Matrix operations** are faster for dense connectivity queries
- **All-pairs algorithms** become more competitive

**Sparse Directed Graphs (E << V²):**
- **Adjacency List** representation is preferred
- **Traversal algorithms** are more efficient
- **Memory usage** is significantly lower

**DAGs (Directed Acyclic Graphs):**
- **Topological sorting** is always possible
- **Dynamic programming** algorithms work efficiently
- **Parallelization** opportunities exist

**Strongly Connected Graphs:**
- **All vertices** reachable from any vertex
- **Single SCC** contains all vertices
- **Diameter** is well-defined

In [9]:
def performance_analysis_directed():
    """
    Analyze the performance characteristics of directed graph algorithms
    """
    V = len(graph.nodes)
    E = len(edges)
    
    print("=== Performance Analysis for Directed Graphs ===")
    print(f"Graph characteristics:")
    print(f"  Vertices (V): {V}")
    print(f"  Directed Edges (E): {E}")
    print(f"  Maximum possible edges: {V * (V - 1)} (complete directed graph)")
    print(f"  Graph density: {E / (V * (V - 1)):.3f}")
    
    # Calculate average degrees
    total_out_degree = sum(len(graph.adj_list[node]) for node in graph.nodes)
    total_in_degree = sum(len(graph.reverse_adj_list[node]) for node in graph.nodes)
    avg_out_degree = total_out_degree / V
    avg_in_degree = total_in_degree / V
    
    print(f"  Average out-degree: {avg_out_degree:.1f}")
    print(f"  Average in-degree: {avg_in_degree:.1f}")
    
    print(f"\nAlgorithm complexity for this graph:")
    
    # Calculate theoretical operations
    traversal_ops = V + E
    scc_ops = V + E  # Both Kosaraju and Tarjan
    topo_ops = V + E
    all_pairs_ops = V**3
    
    print(f"  BFS/DFS Traversal: O(V + E) = O({V} + {E}) = {traversal_ops} operations")
    print(f"  Topological Sort: O(V + E) = {topo_ops} operations")
    print(f"  SCC Algorithms: O(V + E) = {scc_ops} operations")
    print(f"  Cycle Detection: O(V + E) = {traversal_ops} operations")
    print(f"  Single-Source Shortest Path: O(V + E) = {traversal_ops} operations")
    print(f"  All-Pairs Shortest Path: O(V³) = {V}³ = {all_pairs_ops} operations")
    
    # Memory usage comparison
    print(f"\n=== Memory Usage Analysis ===")
    matrix_space = V * V
    list_space = V + 2 * E  # V lists + E outgoing + E incoming edges
    
    print(f"Adjacency Matrix: {matrix_space} entries ({matrix_space * 4} bytes)")
    print(f"Adjacency List: {list_space} entries ({list_space * 8} bytes)")
    print(f"Better choice: {'Matrix' if matrix_space < list_space else 'List'}")
    
    # Algorithm recommendations
    print(f"\n=== Algorithm Recommendations ===")
    print(f"Graph Traversal:")
    print(f"  - Use BFS for shortest hop distance")
    print(f"  - Use DFS for path existence and cycle detection")
    
    print(f"Topological Sorting (if DAG):")
    if V > 1000:
        print(f"  - Use Kahn's algorithm for better cache locality")
    else:
        print(f"  - Both DFS and Kahn's work well for this size")
    
    print(f"Strongly Connected Components:")
    print(f"  - Kosaraju: Simpler implementation, two DFS passes")
    print(f"  - Tarjan: More complex but single pass")
    
    print(f"Shortest Paths:")
    print(f"  - Single-source: BFS ({traversal_ops} ops)")
    if V <= 100:
        print(f"  - All-pairs: Floyd-Warshall acceptable ({all_pairs_ops} ops)")
    else:
        print(f"  - All-pairs: Run BFS from each vertex ({V * traversal_ops} ops)")

def analyze_graph_properties(graph):
    """
    Analyze structural properties of the directed graph
    """
    print(f"\n=== Graph Structure Analysis ===")
    
    # Find sources and sinks
    sources = [node for node in graph.nodes if len(graph.reverse_adj_list[node]) == 0]
    sinks = [node for node in graph.nodes if len(graph.adj_list[node]) == 0]
    
    print(f"Source vertices (in-degree = 0): {sources}")
    print(f"Sink vertices (out-degree = 0): {sinks}")
    
    # Check if graph is DAG
    has_cycle, _ = detect_cycle_dfs(graph)
    is_dag = not has_cycle
    
    print(f"Is DAG (Directed Acyclic Graph): {is_dag}")
    
    # Analyze connectivity
    # Check weak connectivity (treat as undirected)
    def is_weakly_connected():
        # Convert to undirected and check connectivity
        visited = set()
        
        def dfs_undirected(node):
            visited.add(node)
            # Check both outgoing and incoming edges
            for neighbor in graph.adj_list[node]:
                if neighbor not in visited:
                    dfs_undirected(neighbor)
            for neighbor in graph.reverse_adj_list[node]:
                if neighbor not in visited:
                    dfs_undirected(neighbor)
        
        dfs_undirected(graph.nodes[0])
        return len(visited) == len(graph.nodes)
    
    # Check strong connectivity (all nodes reachable from any node)
    def is_strongly_connected():
        # Check if all nodes reachable from first node
        _, _, _, reachable_from_first, _ = bfs_directed(graph, graph.nodes[0])
        if len(reachable_from_first) != len(graph.nodes):
            return False
        
        # Check if first node reachable from all others (using transpose)
        transpose = DirectedGraph(graph.nodes)
        for node in graph.nodes:
            for successor in graph.adj_list[node]:
                transpose.add_edge(successor, node)
        
        _, _, _, reachable_to_first, _ = bfs_directed(transpose, graph.nodes[0])
        return len(reachable_to_first) == len(graph.nodes)
    
    weakly_connected = is_weakly_connected()
    strongly_connected = is_strongly_connected()
    
    print(f"Weakly connected: {weakly_connected}")
    print(f"Strongly connected: {strongly_connected}")
    
    # SCC analysis
    sccs = kosaraju_scc(graph)
    print(f"Number of SCCs: {len(sccs)}")
    
    if len(sccs) == 1:
        print("Graph is strongly connected (single SCC)")
    else:
        print("Graph has multiple SCCs:")
        for i, scc in enumerate(sccs):
            print(f"  SCC {i+1}: {scc}")

# Run comprehensive performance analysis
performance_analysis_directed()
analyze_graph_properties(graph)

# Test algorithm scalability (theoretical)
print(f"\n=== Scalability Analysis ===")
test_sizes = [10, 100, 1000, 10000]

print("Theoretical operation counts for different graph sizes:")
print("Size    V+E        V²        V³")
print("-" * 35)

for V in test_sizes:
    # Assume E ≈ 2V for sparse graph
    E = 2 * V
    ve_ops = V + E
    v2_ops = V * V
    v3_ops = V * V * V
    
    print(f"{V:>4}  {ve_ops:>8}  {v2_ops:>8}  {v3_ops:>10}")

print(f"\nRecommendations by graph size:")
print(f"Small (V < 100): All algorithms practical")
print(f"Medium (V < 1000): Avoid O(V³) algorithms")
print(f"Large (V ≥ 1000): Use sparse representations, avoid all-pairs")

=== Performance Analysis for Directed Graphs ===
Graph characteristics:
  Vertices (V): 7
  Directed Edges (E): 11
  Maximum possible edges: 42 (complete directed graph)
  Graph density: 0.262
  Average out-degree: 1.6
  Average in-degree: 1.6

Algorithm complexity for this graph:
  BFS/DFS Traversal: O(V + E) = O(7 + 11) = 18 operations
  Topological Sort: O(V + E) = 18 operations
  SCC Algorithms: O(V + E) = 18 operations
  Cycle Detection: O(V + E) = 18 operations
  Single-Source Shortest Path: O(V + E) = 18 operations
  All-Pairs Shortest Path: O(V³) = 7³ = 343 operations

=== Memory Usage Analysis ===
Adjacency Matrix: 49 entries (196 bytes)
Adjacency List: 29 entries (232 bytes)
Better choice: List

=== Algorithm Recommendations ===
Graph Traversal:
  - Use BFS for shortest hop distance
  - Use DFS for path existence and cycle detection
Topological Sorting (if DAG):
  - Both DFS and Kahn's work well for this size
Strongly Connected Components:
  - Kosaraju: Simpler implementation

## Summary and Comparison Table

Here's a comprehensive comparison of all algorithms and techniques covered for directed unweighted graphs:

### Algorithm Classification and Use Cases

| Category | Algorithm | Purpose | Key Strength | Limitation |
|----------|-----------|---------|--------------|------------|
| **Graph Traversal** | BFS | Level-order exploration | Shortest hop count | Only explores reachable nodes |
| | DFS | Depth-first exploration | Memory efficient, cycle detection | May not find shortest path |
| **Topological Ordering** | DFS-based Topo Sort | Dependency ordering | Natural with DFS timestamps | Only works on DAGs |
| | Kahn's Algorithm | Dependency ordering | Intuitive, queue-based | Only works on DAGs |
| **Connectivity Analysis** | Kosaraju SCC | Strongly connected components | Simple two-pass approach | Requires transpose graph |
| | Tarjan SCC | Strongly connected components | Single-pass efficiency | More complex implementation |
| **Cycle Detection** | DFS Coloring | Find any cycle | Fast detection | Doesn't find all cycles |
| | All Cycles | Find all simple cycles | Comprehensive analysis | Exponential in worst case |
| **Shortest Path** | BFS Single-Source | Shortest hop distance | Optimal for unweighted | Single source only |
| | BFS All-Pairs | All hop distances | Complete distance matrix | O(V²) space requirement |

### Time Complexity Comparison

| Algorithm | Time Complexity | Space Complexity | Best For |
|-----------|----------------|------------------|----------|
| **BFS/DFS** | O(V + E) | O(V) | Graph exploration, reachability |
| **Topological Sort** | O(V + E) | O(V) | Dependency resolution, scheduling |
| **SCC Algorithms** | O(V + E) | O(V) | Connectivity analysis, graph decomposition |
| **Cycle Detection** | O(V + E) | O(V) | Validation, infinite loop prevention |
| **Shortest Path (Single)** | O(V + E) | O(V) | Distance queries from one source |
| **Shortest Path (All-Pairs)** | O(V³) or O(V(V+E)) | O(V²) | Distance matrix, multiple queries |

### Algorithm Selection Guide

**For Graph Traversal:**
- **Shortest hop distance** → BFS
- **Path existence checking** → DFS
- **Complete reachability analysis** → BFS from each vertex

**For Ordering and Dependencies:**
- **Task scheduling with prerequisites** → Topological Sort (Kahn's or DFS)
- **Build order determination** → Topological Sort
- **Circular dependency detection** → Cycle Detection + Topological Sort

**For Connectivity Analysis:**
- **Finding strongly connected regions** → Kosaraju's or Tarjan's SCC
- **Graph decomposition** → SCC algorithms
- **Mutual reachability** → SCC analysis

**For Cycle Detection:**
- **Simple cycle existence** → DFS with coloring
- **Deadlock detection** → Cycle detection
- **Comprehensive cycle analysis** → All cycles algorithm

**Graph Representation Choice:**
- **Dense graphs (E ≈ V²)** → Adjacency Matrix
- **Sparse graphs (E << V²)** → Adjacency List
- **Frequent predecessor queries** → Adjacency Matrix or Reverse Adjacency List
- **Memory-constrained environments** → Adjacency List

### Special Properties of Directed Graphs

| Property | Implication | Algorithm Impact |
|----------|-------------|------------------|
| **Asymmetric Edges** | u→v doesn't imply v→u | Must track edge directions carefully |
| **In/Out Degrees** | Different in and out degrees | Need separate degree calculations |
| **Strong vs Weak Connectivity** | Two types of connectivity | Different reachability concepts |
| **Cycles** | Can have directed cycles | More complex cycle detection |
| **Topological Order** | Possible only for DAGs | Fundamental for many applications |
| **SCCs** | Natural graph decomposition | Enables divide-and-conquer approaches |

### Performance Recommendations by Graph Size

| Graph Size | Vertices | Recommended Approaches |
|------------|----------|----------------------|
| **Small** | V < 100 | Any algorithm, adjacency matrix OK |
| **Medium** | 100 ≤ V < 1000 | Adjacency list, avoid O(V³) |
| **Large** | V ≥ 1000 | Sparse representations, streaming algorithms |
| **Huge** | V ≥ 100000 | External memory, approximation algorithms |

This completes our comprehensive guide to Directed Unweighted Graphs and their fundamental algorithms!